<a href="https://colab.research.google.com/github/dataengs/create-cube-pyspark/blob/main/create_cube_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pyspark

In [4]:
#importa bibliotecas
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [5]:
#cria sessao spark
spark = SparkSession.builder.appName('CreateMyCube').getOrCreate()

In [6]:
data = [("Padaria", "Marcos", 0), 
        ("Bebidas", "Sebastian", 3),
        ("Bebidas", "Marcos", 2),
        ("Hortifruti", "Fred", 10),
        ("Higiene pessoal", "Fred", 2),
        ("Padaria", "Fred", 0), 
        ("Bebidas", "Silva", 5), 
        ("Hortifruti", "Sebastian", 7)
       ]

#cria um schema manual para criar o dataframe
schema = StructType([
            StructField("Categoria", StringType(), True),
            StructField("Vendedor", StringType(), True),
            StructField("QtdVendas", IntegerType(), True)
        ])

#cria um spark dataframe 
df = spark.createDataFrame(data=data,schema=schema)
#exibe schema
df.printSchema()
#exibe os dados de forma vertical
df.show(truncate=False)

root
 |-- Categoria: string (nullable = true)
 |-- Vendedor: string (nullable = true)
 |-- QtdVendas: integer (nullable = true)

+---------------+---------+---------+
|Categoria      |Vendedor |QtdVendas|
+---------------+---------+---------+
|Padaria        |Marcos   |0        |
|Bebidas        |Sebastian|3        |
|Bebidas        |Marcos   |2        |
|Hortifruti     |Fred     |10       |
|Higiene pessoal|Fred     |2        |
|Padaria        |Fred     |0        |
|Bebidas        |Silva    |5        |
|Hortifruti     |Sebastian|7        |
+---------------+---------+---------+



In [7]:
#Aplicando a função cube, temos o seguinte resultado
df.cube("Categoria", "Vendedor") \
  .sum() \
  .orderBy("Categoria", "Vendedor") \
  .show()

+---------------+---------+--------------+
|      Categoria| Vendedor|sum(QtdVendas)|
+---------------+---------+--------------+
|           null|     null|            29|
|           null|     Fred|            12|
|           null|   Marcos|             2|
|           null|Sebastian|            10|
|           null|    Silva|             5|
|        Bebidas|     null|            10|
|        Bebidas|   Marcos|             2|
|        Bebidas|Sebastian|             3|
|        Bebidas|    Silva|             5|
|Higiene pessoal|     null|             2|
|Higiene pessoal|     Fred|             2|
|     Hortifruti|     null|            17|
|     Hortifruti|     Fred|            10|
|     Hortifruti|Sebastian|             7|
|        Padaria|     null|             0|
|        Padaria|     Fred|             0|
|        Padaria|   Marcos|             0|
+---------------+---------+--------------+

